In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import pymysql
import os
from dotenv import load_dotenv

In [2]:
# load environment variables from .env file
load_dotenv() 
password=os.environ.get('PyMySQL_PASSWORD')
db=os.environ.get('PyMySQL_db')


#connect to my local database in mysql
connection = pymysql.connect(host="localhost",
                             user="root",
                             password=password,
                             db=db,
                             port=3306,
                             autocommit=True)

### (1) In order to show the company order volume per overall website session growth by quarter.  I have pulled overall session and order volume, trended by quarter for the life of the business

In [3]:
query1 ='''
SELECT
	YEAR(w.created_at) AS yr,
	quarter(w.created_at) AS quarter,
	COUNT(DISTINCT w.website_session_id) AS overall_session,
    COUNT(DISTINCT o.website_session_id) AS order_volume
FROM
	website_sessions w
LEFT JOIN
	orders o
    ON w.website_session_id = o.website_session_id
GROUP BY
	YEAR(w.created_at),quarter(w.created_at);
'''
df1 = pd.read_sql_query(query1,con = connection)

In [4]:
df1

,yr,quarter,overall_session,order_volume
0,2012,1,1879,60
1,2012,2,11433,347
2,2012,3,16892,684
3,2012,4,32266,1495
4,2013,1,19833,1273
5,2013,2,24745,1718
6,2013,3,27663,1840
7,2013,4,40540,2616
8,2014,1,46779,3069
9,2014,2,53129,3848


### (2) Next, let's showcase all four efficiency improvements.  I've here shown quarterly figures since we launched, for session-to-order conversion rate, revenue per order and revenue per session

In [5]:
query2 ='''
SELECT
	YEAR(w.created_at) AS yr,
	quarter(w.created_at) AS quarter,
    COUNT(DISTINCT o.website_session_id)/COUNT(DISTINCT w.website_session_id) AS conv_rate,
    SUM(o.price_usd)/ COUNT(DISTINCT o.website_session_id) AS revenue_per_order,
    SUM(o.price_usd)/ COUNT(DISTINCT w.website_session_id) AS revenue_per_session
FROM
	website_sessions w
LEFT JOIN
	orders o
    ON w.website_session_id = o.website_session_id
GROUP BY
	YEAR(w.created_at),quarter(w.created_at);
'''
df2 = pd.read_sql_query(query2,con = connection)

In [6]:
df2

,yr,quarter,conv_rate,revenue_per_order,revenue_per_session
0,2012,1,0.0319,49.990000,1.596275
1,2012,2,0.0304,49.990000,1.517233
2,2012,3,0.0405,49.990000,2.024222
3,2012,4,0.0463,49.990000,2.316217
4,2013,1,0.0642,52.142396,3.346809
5,2013,2,0.0694,51.538312,3.578211
6,2013,3,0.0665,51.734533,3.441114
7,2013,4,0.0645,54.715688,3.530741
8,2014,1,0.0656,62.160684,4.078136
9,2014,2,0.0724,64.374207,4.662462


### (3)This has shown how the company grown specific channels. I have pulled a quarterly view of orders from Gsearch nonbrand, Bsearch nonbrand, brand search overall, organic search, and direct type-in

In [7]:
query3 ='''
SELECT
	YEAR(orders_by_specific_channels.created_at) AS yr,
    quarter(orders_by_specific_channels.created_at) AS quarter ,
    COUNT(DISTINCT CASE WHEN channels = "gsearch_nonbrand" THEN website_session_id ELSE NULL END) AS gsearch_nonbrand,
    COUNT(DISTINCT CASE WHEN channels = "bsearch_nonbrand" THEN website_session_id ELSE NULL END) AS bsearch_nonbrand,
    COUNT(DISTINCT CASE WHEN channels = "brand_search" THEN website_session_id ELSE NULL END) AS brand_search,
    COUNT(DISTINCT CASE WHEN channels = "organic_search" THEN website_session_id ELSE NULL END) AS organic_search,
	COUNT(DISTINCT CASE WHEN channels = "direct_type_in" THEN website_session_id ELSE NULL END) AS direct_type_in
FROM
(SELECT
	w.created_at,
	-- quarter(w.created_at) AS quarter,
    o.website_session_id,
    CASE
		WHEN utm_source = 'gsearch' AND utm_campaign = "nonbrand" THEN "gsearch_nonbrand"
		WHEN utm_source = 'bsearch' AND utm_campaign = "nonbrand" THEN "bsearch_nonbrand"
		WHEN utm_campaign = "brand" THEN "brand_search"
		WHEN utm_source IS NULL AND utm_campaign IS NULL AND  http_referer IS NOT NULL THEN  "organic_search"
		WHEN utm_source IS NULL AND utm_campaign IS NULL AND  http_referer IS NULL THEN "direct_type_in"
	ELSE "others" END AS channels
FROM
	website_sessions w
LEFT JOIN
	orders o
    ON w.website_session_id = o.website_session_id) AS orders_by_specific_channels
GROUP BY
	YEAR(orders_by_specific_channels.created_at),quarter(orders_by_specific_channels.created_at)
ORDER BY
	YEAR(orders_by_specific_channels.created_at), quarter(orders_by_specific_channels.created_at);
'''
df3 = pd.read_sql_query(query3,con = connection)

In [8]:
df3 # order volume by channels

,yr,quarter,gsearch_nonbrand,bsearch_nonbrand,brand_search,organic_search,direct_type_in
0,2012,1,60,0,0,0,0
1,2012,2,291,0,20,15,21
2,2012,3,482,82,48,40,32
3,2012,4,913,311,88,94,89
4,2013,1,766,183,108,125,91
5,2013,2,1114,237,114,134,119
6,2013,3,1132,245,153,167,143
7,2013,4,1657,291,248,223,197
8,2014,1,1667,344,354,338,311
9,2014,2,2208,427,410,436,367


### (4)Next, let's show the overall session-to-order conversion rate trends for those same channels, by quarter.

In [9]:
try:
    query4i ='''
    CREATE TEMPORARY TABLE sessions_and_orders_by_specific_channels
    SELECT
        w.created_at,
        -- quarter(w.created_at) AS quarter,
        o.website_session_id AS order_session_id,
        w.website_session_id AS all_session_id,
        CASE
            WHEN utm_source = 'gsearch' AND utm_campaign = "nonbrand" THEN "gsearch_nonbrand"
            WHEN utm_source = 'bsearch' AND utm_campaign = "nonbrand" THEN "bsearch_nonbrand"
            WHEN utm_campaign = "brand" THEN "brand_search"
            WHEN utm_source IS NULL AND utm_campaign IS NULL AND  http_referer IS NOT NULL THEN  "organic_search"
            WHEN utm_source IS NULL AND utm_campaign IS NULL AND  http_referer IS NULL THEN "direct_type_in"
        ELSE "others" END AS channels
    FROM
        website_sessions w
    LEFT JOIN
        orders o
        ON w.website_session_id = o.website_session_id;
    ''' 
    pd.read_sql_query(query4i,con = connection)

except Exception:
    pass
finally:
    query4i ='''
    SELECT
        w.created_at,
        -- quarter(w.created_at) AS quarter,
        o.website_session_id AS order_session_id,
        w.website_session_id AS all_session_id,
        CASE
            WHEN utm_source = 'gsearch' AND utm_campaign = "nonbrand" THEN "gsearch_nonbrand"
            WHEN utm_source = 'bsearch' AND utm_campaign = "nonbrand" THEN "bsearch_nonbrand"
            WHEN utm_campaign = "brand" THEN "brand_search"
            WHEN utm_source IS NULL AND utm_campaign IS NULL AND  http_referer IS NOT NULL THEN  "organic_search"
            WHEN utm_source IS NULL AND utm_campaign IS NULL AND  http_referer IS NULL THEN "direct_type_in"
        ELSE "others" END AS channels
    FROM
        website_sessions w
    LEFT JOIN
        orders o
        ON w.website_session_id = o.website_session_id;
    ''' 
    pd.read_sql_query(query4i,con = connection)

query4ii ='''
SELECT
	YEAR(created_at) AS year,
    quarter(created_at) AS quarter ,
	COUNT(DISTINCT CASE WHEN channels = "gsearch_nonbrand" THEN order_session_id ELSE NULL END) /COUNT(DISTINCT CASE WHEN channels = "gsearch_nonbrand" THEN all_session_id ELSE NULL END) AS gsearch_nonbrand_cr,
    COUNT(DISTINCT CASE WHEN channels = "bsearch_nonbrand" THEN order_session_id ELSE NULL END) / COUNT(DISTINCT CASE WHEN channels = "bsearch_nonbrand" THEN all_session_id ELSE NULL END) AS bsearch_nonbrand_cr,
    COUNT(DISTINCT CASE WHEN channels = "brand_search" THEN order_session_id ELSE NULL END)/COUNT(DISTINCT CASE WHEN channels = "brand_search" THEN all_session_id ELSE NULL END) AS brand_search_cr,
    COUNT(DISTINCT CASE WHEN channels = "organic_search" THEN order_session_id ELSE NULL END)/COUNT(DISTINCT CASE WHEN channels = "organic_search" THEN all_session_id ELSE NULL END) AS organic_search_cr,
	COUNT(DISTINCT CASE WHEN channels = "direct_type_in" THEN order_session_id ELSE NULL END)/COUNT(DISTINCT CASE WHEN channels = "direct_type_in" THEN all_session_id ELSE NULL END) AS direct_type_in_cr
FROM
	sessions_and_orders_by_specific_channels
GROUP BY
	YEAR(created_at),quarter(created_at)
ORDER BY
	YEAR(created_at), quarter(created_at);
'''
df4 = pd.read_sql_query(query4ii,con = connection)

In [10]:
df4 #conversion rate by channels

,year,quarter,gsearch_nonbrand_cr,bsearch_nonbrand_cr,brand_search_cr,organic_search_cr,direct_type_in_cr
0,2012,1,0.0324,NaN,0.0000,0.0000,0.0000
1,2012,2,0.0284,NaN,0.0526,0.0359,0.0536
2,2012,3,0.0384,0.0408,0.0602,0.0498,0.0443
3,2012,4,0.0436,0.0497,0.0531,0.0539,0.0537
4,2013,1,0.0612,0.0693,0.0703,0.0753,0.0614
5,2013,2,0.0685,0.0690,0.0679,0.0760,0.0735
6,2013,3,0.0639,0.0697,0.0703,0.0734,0.0719
7,2013,4,0.0629,0.0601,0.0801,0.0694,0.0647
8,2014,1,0.0693,0.0704,0.0839,0.0756,0.0765
9,2014,2,0.0702,0.0695,0.0804,0.0797,0.0738


### (5)We’ve come a long way since the days of selling a single product. Let’s pull monthly trending for revenue and margin by product, along with total sales and revenue. Note anything you notice about seasonality.

In [11]:
query5 ='''
SELECT
	YEAR(created_at) AS year,
    MONTH(created_at) AS month,
    SUM(price_usd) AS total_sales,
    SUM(CASE WHEN product_id = 1 THEN price_usd ELSE 0 END) AS sales_prod_1,
	SUM(CASE WHEN product_id = 2 THEN price_usd ELSE 0 END) AS sales_prod_2,
    SUM(CASE WHEN product_id = 3 THEN price_usd ELSE 0 END) AS sales_prod_3,
    SUM(CASE WHEN product_id = 4 THEN price_usd ELSE 0 END) AS sales_prod_4,
	SUM(CASE WHEN product_id = 1 THEN price_usd - cogs_usd ELSE 0 END) AS margin_prod_1,
	SUM(CASE WHEN product_id = 2 THEN price_usd - cogs_usd ELSE 0 END) AS margin_prod_2,
    SUM(CASE WHEN product_id = 3 THEN price_usd - cogs_usd ELSE 0 END) AS margin_prod_3,
    SUM(CASE WHEN product_id = 4 THEN price_usd - cogs_usd ELSE 0 END) AS margin_prod_4

FROM
order_items
GROUP BY
	YEAR(created_at),
    MONTH(created_at) ;

'''
df5 = pd.read_sql_query(query5,con = connection)

In [12]:
df5 # revenue and margin by products 

,year,month,total_sales,sales_prod_1,sales_prod_2,sales_prod_3,sales_prod_4,margin_prod_1,margin_prod_2,margin_prod_3,margin_prod_4
0,2012,3,2999.40,2999.40,0.00,0.00,0.00,1830.0,0.0,0.0,0.0
1,2012,4,4949.01,4949.01,0.00,0.00,0.00,3019.5,0.0,0.0,0.0
2,2012,5,5398.92,5398.92,0.00,0.00,0.00,3294.0,0.0,0.0,0.0
3,2012,6,6998.60,6998.60,0.00,0.00,0.00,4270.0,0.0,0.0,0.0
4,2012,7,8448.31,8448.31,0.00,0.00,0.00,5154.5,0.0,0.0,0.0
5,2012,8,11397.72,11397.72,0.00,0.00,0.00,6954.0,0.0,0.0,0.0
6,2012,9,14347.13,14347.13,0.00,0.00,0.00,8753.5,0.0,0.0,0.0
7,2012,10,18546.29,18546.29,0.00,0.00,0.00,11315.5,0.0,0.0,0.0
8,2012,11,30893.82,30893.82,0.00,0.00,0.00,18849.0,0.0,0.0,0.0
9,2012,12,25294.94,25294.94,0.00,0.00,0.00,15433.0,0.0,0.0,0.0


### (6) Let’s dive deeper into the impact of introducing new products. Please pull monthly sessions to  the /products page, and show how the % of those sessions clicking through another page has changed over time, along with a view of how conversion from /products to placing an order has improved.

In [13]:
try:       
    query6i ='''
    CREATE TEMPORARY TABLE products_pageviews
    SELECT
        website_session_id, 
        website_pageview_id, 
        created_at AS saw_product_page_at

    FROM website_pageviews 
    WHERE pageview_url = '/products';
    '''
    pd.read_sql_query(query6i,con = connection)
    
except Exception:
    pass

finally:
    query6i ='''
    SELECT
        website_session_id, 
        website_pageview_id, 
        created_at AS saw_product_page_at

    FROM website_pageviews 
    WHERE pageview_url = '/products';
    '''
    pd.read_sql_query(query6i,con = connection)

query6ii ='''
SELECT 
	YEAR(saw_product_page_at) AS yr, 
    MONTH(saw_product_page_at) AS mo,
    COUNT(DISTINCT products_pageviews.website_session_id) AS sessions_to_product_page, 
    COUNT(DISTINCT website_pageviews.website_session_id) AS clicked_to_next_page, 
    COUNT(DISTINCT website_pageviews.website_session_id)/COUNT(DISTINCT products_pageviews.website_session_id) AS clickthrough_rt,
    COUNT(DISTINCT orders.order_id) AS orders,
    COUNT(DISTINCT orders.order_id)/COUNT(DISTINCT products_pageviews.website_session_id) AS products_to_order_rt
FROM products_pageviews
	LEFT JOIN website_pageviews 
		ON website_pageviews.website_session_id = products_pageviews.website_session_id -- same session
        AND website_pageviews.website_pageview_id > products_pageviews.website_pageview_id -- they had another page AFTER
	LEFT JOIN orders 
		ON orders.website_session_id = products_pageviews.website_session_id
GROUP BY 1,2
;
'''
df6 = pd.read_sql_query(query6ii,con = connection)

In [14]:
df6 #website sessions to product page clicked through rate and product to order page conversion rate

,yr,mo,sessions_to_product_page,clicked_to_next_page,clickthrough_rt,orders,products_to_order_rt
0,2012,3,743,530,0.7133,60,0.0808
1,2012,4,1447,1029,0.7111,99,0.0684
2,2012,5,1584,1135,0.7165,108,0.0682
3,2012,6,1752,1247,0.7118,140,0.0799
4,2012,7,2018,1438,0.7126,169,0.0837
5,2012,8,3012,2198,0.7297,228,0.0757
6,2012,9,3126,2258,0.7223,287,0.0918
7,2012,10,4030,2948,0.7315,371,0.0921
8,2012,11,6743,4849,0.7191,618,0.0917
9,2012,12,5013,3620,0.7221,506,0.1009


### (7) We made our 4th product available as a primary product on December 05, 2014 (it was previously only a cross-sell item). Could you please pull sales data since then, and show how well each product cross-sells from one another?


In [15]:
try:
    query7i ='''
    CREATE TEMPORARY TABLE primary_products
    SELECT 
        order_id, 
        primary_product_id, 
        created_at AS ordered_at
    FROM orders 
    WHERE created_at > '2014-12-05' 
    ;

    '''
    pd.read_sql_query(query7i,con = connection)
except Exception:
    pass
finally:
    query7i ='''
    SELECT 
        order_id, 
        primary_product_id, 
        created_at AS ordered_at
    FROM orders 
    WHERE created_at > '2014-12-05' 
    ;

    '''
    pd.read_sql_query(query7i,con = connection)

In [16]:
query7ii ='''

SELECT 
	primary_product_id, 
    COUNT(DISTINCT order_id) AS total_orders, 
    COUNT(DISTINCT CASE WHEN cross_sell_product_id = 1 THEN order_id ELSE NULL END) AS x_sold_p1,
    COUNT(DISTINCT CASE WHEN cross_sell_product_id = 2 THEN order_id ELSE NULL END) AS x_sold_p2,
    COUNT(DISTINCT CASE WHEN cross_sell_product_id = 3 THEN order_id ELSE NULL END) AS x_sold_p3,
    COUNT(DISTINCT CASE WHEN cross_sell_product_id = 4 THEN order_id ELSE NULL END) AS x_sold_p4,
    COUNT(DISTINCT CASE WHEN cross_sell_product_id = 1 THEN order_id ELSE NULL END)/COUNT(DISTINCT order_id) AS p1_xsell_rt,
    COUNT(DISTINCT CASE WHEN cross_sell_product_id = 2 THEN order_id ELSE NULL END)/COUNT(DISTINCT order_id) AS p2_xsell_rt,
    COUNT(DISTINCT CASE WHEN cross_sell_product_id = 3 THEN order_id ELSE NULL END)/COUNT(DISTINCT order_id) AS p3_xsell_rt,
    COUNT(DISTINCT CASE WHEN cross_sell_product_id = 4 THEN order_id ELSE NULL END)/COUNT(DISTINCT order_id) AS p4_xsell_rt
FROM
(
SELECT
	primary_products.*, 
    order_items.product_id AS cross_sell_product_id
FROM primary_products
	LEFT JOIN order_items 
		ON order_items.order_id = primary_products.order_id
        AND order_items.is_primary_item = 0 -- only bringing in cross-sells
) AS primary_w_cross_sell
GROUP BY 1;
'''
df7 = pd.read_sql_query(query7ii,con = connection)

In [17]:
df7 #cross-sell of all products from one another

,primary_product_id,total_orders,x_sold_p1,x_sold_p2,x_sold_p3,x_sold_p4,p1_xsell_rt,p2_xsell_rt,p3_xsell_rt,p4_xsell_rt
0,1,4467,0,238,553,933,0.0000,0.0533,0.1238,0.2089
1,2,1277,25,0,40,260,0.0196,0.0000,0.0313,0.2036
2,3,929,84,40,0,208,0.0904,0.0431,0.0000,0.2239
3,4,581,16,9,22,0,0.0275,0.0155,0.0379,0.0000
